# H_22082024
# Analisis del volumen profile para encontrar zonas de baja cotización (FVG like) y como se comporta el precio en esas zonas.

En este Python Notebook analizaremos, con datos de mt5, como se comporta el mercado en las zonas previamente definidas.

## Conceptos clave del perfil de mercado:

1. **Oportunidad de precio en el tiempo (TPO):**
    
    representa los niveles de precio negociados durante intervalos de tiempo específicos. En un gráfico de perfil de mercado, las TPO indican la frecuencia con la que se negoció cada nivel de precio durante una sesión.
    
2. **Área de valor:**
    
    rango de precios en el que se produjo el 70 % de la actividad comercial durante una sesión. Esta área representa el valor justo percibido del mercado.
    
3. **Punto de control (POC):**
    
    el nivel de precio con el mayor volumen durante la sesión. El POC suele ser un nivel clave de soporte o resistencia.
    
4. **Balance inicial (IB):**
    
    rango de precios negociados durante la primera hora de la sesión de negociación. Proporciona una visión anticipada de la estructura del mercado del día.

In [117]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
import pytz
from datetime import datetime
import matplotlib.pyplot as plt
import json
import plotly.express as px
import plotly.graph_objects as go

In [118]:
# Variables

TICKER = "SI"
STD_MULTIPLIER = 1
minute_data = "CL_V_M1_202105250000_202408012359.csv"
daily_data = "CL_V_Daily_202105250000_202408010000.csv"

## 1. Definición de zonas de baja cotización para posterior analisis.
El primer paso a dar es definir como encontraremos las **zonas de baja cotización** en una sesion de mercado de forma objetiva.

A continuación, definiremos que es una zona de baja cotización para nuestro estudio:

1. Analisis basado en el *volumen profile*, el cual analiza el numero de cotizaciónes por precio que tiene lugar en una sesión de trading predefinida (en nuestro estudio, una sesión será un dia 1D).
2. La media y la StdDev del volumen profile de la sesión nos ayudarán a analizar correctamente las zonas.

Por tanto, una **zona de baja cotización** consiste en:
- Los niveles de precio que se encuentran N*StdDev(mean(volume_profile)) por debajo.

In [119]:
def data_from_mt5():
    # connect to MetaTrader 5
    if not mt5.initialize():
        print("initialize() failed")
        mt5.shutdown()

    # set time zone to UTC
    timezone = pytz.timezone("Etc/UTC")
    # create 'datetime' objects in UTC time zone to avoid the implementation of a local time zone offset
    utc_from = datetime(2010, 1, 1, tzinfo=timezone)
    utc_to = datetime(2024, 8, 1, tzinfo=timezone)

    # request AUDUSD ticks within 11.01.2020 - 11.01.2020
    ohlcv = mt5.copy_rates_range(TICKER, mt5.TIMEFRAME_M1, utc_from, utc_to)
    daily_ohlcv = mt5.copy_rates_range(TICKER, mt5.TIMEFRAME_D1, utc_from, utc_to)

    mt5.shutdown()

    df = pd.DataFrame(ohlcv)
    df['time']=pd.to_datetime(df['time'], unit='s')

    # adaptamos el dataframe D1 para luego hacer los analisis de las sesiones
    daily_df = pd.DataFrame(daily_ohlcv)
    daily_df['time']=pd.to_datetime(daily_df['time'], unit='s')

    df = df.set_index('time')
    del df['real_volume']
    del df['spread']

    daily_df = daily_df.set_index('time')
    del daily_df['real_volume']
    del daily_df['spread']

    return df, daily_df

def data_from_csv():
    df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/'+minute_data, sep='\t')
    df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    del df['<TIME>']
    del df['<VOL>']
    del df['<SPREAD>']
    df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    df = df.set_index('time')

    daily_df = pd.read_csv('C:/Users/iamfr/AlgoTrading/DATA/'+daily_data, sep='\t')
    #df['<DATE>'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
    #del df['<TIME>']
    del daily_df['<VOL>']
    del daily_df['<SPREAD>']
    daily_df.columns = ['time', 'open','high', 'low', 'close', 'tick_volume']
    daily_df = daily_df.set_index('time')
    #daily_df = df.resample('1B').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last', 'tick_volume': 'sum'})

    return df, daily_df

df, daily_df = data_from_csv()

df

,open,high,low,close,tick_volume
time,,,,,
2021-05-25 00:00:00,64.76,64.76,64.17,64.17,2
2021-05-25 00:40:00,64.17,64.17,64.17,64.17,1
2021-05-25 01:00:00,64.73,64.78,64.71,64.73,24
2021-05-25 01:01:00,64.73,64.73,64.70,64.70,9
2021-05-25 01:02:00,64.70,64.71,64.70,64.70,5
...,...,...,...,...,...
2024-08-01 23:55:00,75.89,75.92,75.89,75.91,11
2024-08-01 23:56:00,75.92,75.92,75.89,75.90,10
2024-08-01 23:57:00,75.90,75.92,75.89,75.91,8


In [120]:
def create_market_profile(data, getPOC=True):
    profile = data.groupby('close')['tick_volume'].sum().reset_index()
    total_volume = profile['tick_volume'].sum()
    profile['volume_cumsum'] = profile['tick_volume'].cumsum()

    value_area_cutoff = total_volume * 0.70
    value_area_df = profile[profile['volume_cumsum'] <= value_area_cutoff]
    POC = 0
    if getPOC:
        POC = profile.loc[profile['tick_volume'].idxmax(), 'close']
    else:
        POC = profile.loc[profile['tick_volume'].idxmin(), 'close']

    return profile, value_area_df, POC

def plot_market_profile(profile, value_area_df, POC):
    plt.figure(figsize=(10, 6))
    plt.barh(profile['close'], profile['tick_volume'], color='blue', edgecolor='black')
    plt.barh(value_area_df['close'], value_area_df['tick_volume'], color='green', edgecolor='black')
    plt.axhline(POC, color='red', linestyle='--', label=f'POC: {POC}')

    plt.xlabel('Volume')
    plt.ylabel('Price')
    plt.title(f'Market Profile')
    plt.legend()
    plt.show()

In [121]:
def get_no_fair_range_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] < threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

def get_max_vol_zone(df, threshold):
    # Crear una máscara booleana para identificar dónde 'tick_volume' es inferior al umbral
    df.sort_values(by=['close'])
    mask = df['tick_volume'] > threshold

    # Encontrar los índices donde empieza y termina cada zona
    rangos = []
    inicio = None
    rsize = 0
    rango_max = []

    for i in range(len(df)):
        if mask[i]:
            if inicio is None:  # Se inicia una nueva zona
                inicio = i
        else:
            if inicio is not None:  # Se cierra la zona actual
                rangos.append([inicio, i - 1])
                inicio = None

    # Si la última zona no se cierra explícitamente en el bucle
    if inicio is not None:
        rangos.append([inicio, len(df) - 1])
                 
    for rango in rangos:
        if rsize <= (rango[1] - rango[0]):  
              rsize = rango[1] - rango[0]
              rango_max = rango

    if len(rangos) < 1:
        return False
    return [df.loc[rango_max[0], 'close'], df.loc[rango_max[1], 'close']]

In [122]:
# iteramos cada dia, y cada zona
def get_no_fair_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, MIN = create_market_profile(day, False)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        no_fair_value_zone = get_no_fair_range_zone(profile, threshold)

        if no_fair_value_zone == False:
            output = {
                "time": index1,
                "min_zone_high": np.nan,
                "min_zone_low": np.nan,
                "MIN": np.nan,
            }
            out.append(output)
        else:
            output = {
                "time": index1,
                "min_zone_high": no_fair_value_zone[1],
                "min_zone_low": no_fair_value_zone[0],
                "MIN": MIN,
            }
            out.append(output)

    return out

# iteramos cada dia, y cada zona
def get_max_vol_zone_by_day(df):
    out = []

    for index1, day in df.groupby(df.index.date):

        profile, value_area_df, POC = create_market_profile(day)
        mean = profile['tick_volume'].mean()
        stddev = profile['tick_volume'].std()
        threshold = mean + stddev * STD_MULTIPLIER

        max_value_zone = get_max_vol_zone(profile, threshold)

        if max_value_zone == False:
            output = {
                "time": index1,
                "max_zone_high": np.nan,
                "max_zone_low": np.nan,
                "POC": np.nan,
            }
            out.append(output)
        else:
            output = {
                "time": index1,
                "max_zone_high": max_value_zone[1],
                "max_zone_low": max_value_zone[0],
                "POC": POC,
            }
            out.append(output)

    return out

## Analisis de datos

A partir de aqui, ya disponemos de las herramientas de manipulación de los datos, como para poder empezar a analizarlos.

Disponemos de una lista "ZONES_BY_DAY", la qual nos permitirá referenciar las zonas de baja cotización por dia, y sus niveles de precio. Asi podemos hacer un analisis más comodo.

In [123]:
#zones = get_no_fair_zone_by_day(df)
max_zones = get_max_vol_zone_by_day(df)
df_max_zones = pd.DataFrame(max_zones)
#df_zones['time'].astype('datetime64[ns]')
df_max_zones = df_max_zones.set_index('time')
df_max_zones.index.astype('datetime64[ns]')

#zones = get_no_fair_zone_by_day(df)
min_zones = get_no_fair_zone_by_day(df)
df_min_zones = pd.DataFrame(min_zones)
#df_zones['time'].astype('datetime64[ns]')
df_min_zones = df_min_zones.set_index('time')
df_min_zones.index.astype('datetime64[ns]')

main_df_2 = pd.merge(df_max_zones, df_min_zones, on=daily_df.index)
main_df_2 = main_df_2.set_index('key_0')
main_df = pd.merge(daily_df, main_df_2, on=daily_df.index)
#daily_df.join(df_zones, on=daily_df.index , how='inner')
main_df = main_df.set_index('key_0')
main_df.dropna()
main_df

#main_df.to_csv( TICKER+'_H22082024_Output_Data.csv', index=True)

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN
key_0,,,,,,,,,,,
2021.05.25,64.76,65.05,64.16,64.70,20950,64.91,64.86,64.87,64.63,64.17,64.21
2021.05.26,62.25,65.11,62.25,64.93,19551,64.81,64.75,64.91,64.58,62.25,62.25
2021.05.27,64.77,65.59,64.20,65.54,16575,64.51,64.48,65.28,65.27,64.79,64.20
2021.05.28,65.38,66.11,64.70,65.29,20234,65.65,65.61,65.63,65.56,64.70,64.70
2021.05.31,64.50,66.06,64.50,65.25,11688,65.75,65.73,65.74,65.58,65.41,64.50
...,...,...,...,...,...,...,...,...,...,...,...
2024.07.26,77.14,77.40,75.07,75.30,24615,77.07,77.04,75.51,77.03,76.30,75.77
2024.07.29,76.19,76.52,74.34,74.86,28905,74.81,74.75,76.04,75.91,74.86,76.51
2024.07.30,74.85,74.92,73.74,74.34,27580,74.00,73.95,73.96,74.90,74.40,74.84


### SETUP_1. Cierre de rango en la siguiente sesión

Analizaremos si en la siguiente sesión, el precio cierra el rango del dia anterior.

In [124]:
def num1_next_session_close_zone(df):
    results = []

    for i in range(len(df.index)-1):
        if i == 0: pass
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']

        # Comprobar si se cumple la condición
        if zone_high <= high and zone_high >= low and zone_low >= low and zone_low <= high:
            results.append(True)  # Cumple la condición
        else:
            results.append(False)  # No cumple la condición

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_1'] = results

    return df['SETUP_1'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de cierre de la zona: ", num1_next_session_close_zone(main_df))

Porcentaje de cierre de la zona:  SETUP_1
True     64.91002570694087%
False    35.08997429305913%
Name: proportion, dtype: object


### SETUP_2. Cierre de medio rango en la siguiente sesion

Analizaremos si en la siguiente sesión, el precio cierra la midat del rango.

In [125]:
def num2_next_session_close_half_zone(df):
    results = []

    for i in range(len(df.index) - 1):
        if i == 0: pass
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']
        zone_mid = ((zone_high - zone_low) / 2) + zone_low

        if apertura >= zone_high:
            # Mitad superior
            if zone_high <= high and zone_high >= low and zone_mid >= low and zone_mid <= high:
                results.append(True)
            else:
                results.append(False)
        elif apertura <= zone_low:
            # Mitad inferior
            if zone_mid <= high and zone_mid >= low and zone_low >= low and zone_low <= high:
                results.append(True)
            else:
                results.append(False)
        else:
            results.append(False)
    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_2'] = results

    return df['SETUP_2'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de cierre de media zona: ", num2_next_session_close_half_zone(main_df))

Porcentaje de cierre de media zona:  SETUP_2
True     61.696658097686374%
False    38.303341902313626%
Name: proportion, dtype: object


### SETUP_3. Desde apertura a extremo de zona

Analisis de la cantidad de veces que el precio recorre el rango entre la apertura de la sesion y un valor extremo.

Depende de si la apertura es por debajo o per encima de la zona de alta capitalización, buscaremos largos o cortos.

In [126]:
def num3_from_open_to_range_max_volume(df):
    results = []

    for i in range(len(df.index) - 1):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        zone_high = df.iloc[i]['max_zone_high']
        zone_low = df.iloc[i]['max_zone_low']

        if apertura >= zone_high:
            # Bajista
            if zone_low <= high and zone_low >= low:
                results.append(True)
            else:
                results.append(False)
        elif apertura <= zone_low:
            # Alcista
            if zone_high <= high and zone_high >= low:
                results.append(True)
            else:
                results.append(False)
        else: 
            results.append(False)

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_3'] = results

    return df['SETUP_3'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de Apertura-Extremo: ", num3_from_open_to_range_max_volume(main_df))

Porcentaje de Apertura-Extremo:  SETUP_3
True     60.411311053984576%
False    39.588688946015424%
Name: proportion, dtype: object


### SETUP_4. Recorrido a POC anterior session desde apertura 

Analisis de la cantidad de veces que se testea el valor POC de la anterior sesion.

In [127]:
def num4_POC_test(df):
    results = []

    for i in range(len(df.index) - 1):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        poc = df.iloc[i]['POC']

        if apertura >= poc:
            # Bajista
            if poc <= high and poc >= low:
                results.append(True)
            else:
                results.append(False)

        elif apertura <= poc:
            # Alcista
            if poc <= high and poc >= low:
                results.append(True)
            else:
                results.append(False)
        else: 
            results.append(False)

    # Agregar un 0 adicional al final para igualar el tamaño de la columna con el dataframe original
    results.append(False)

    # Crear una nueva columna en el dataframe con los resultados
    df['SETUP_4'] = results

    return df['SETUP_4'].value_counts(normalize=True).mul(100).astype(str)+'%'

print("Porcentaje de test POC: ", num4_POC_test(main_df))

Porcentaje de test POC:  SETUP_4
True     69.02313624678663%
False    30.97686375321337%
Name: proportion, dtype: object


### Nivel de mayor probabilidad estadistica > SETUP_4

A continuación, sacaremos más estadisticas que complementarán el estudio.

In [128]:
def cummulative_not_setup4_true(df):
    results = []
    tmp = 0

    for index, row in df.iterrows():
        if row['SETUP_4'] == False:
            tmp = tmp + 1
        else:
            results.append(tmp)
            tmp = 0

    nparr = np.array(results)

    return nparr.max()

print("Maximos fallos consecutivos de SETUP 4: ", cummulative_not_setup4_true(main_df))

Maximos fallos consecutivos de SETUP 4:  6


In [129]:
def setup4_range_stddev(df):
    df['SETUP_4_range'] = abs(df['POC'] - df['open'])

    true_df = df.query('SETUP_4 == True')
    tmean = true_df['SETUP_4_range'].mean()
    tstddev = true_df['SETUP_4_range'].std()

    false_df = df.query('SETUP_4 == False')
    fmean = false_df['SETUP_4_range'].mean()
    fstddev = false_df['SETUP_4_range'].std()

    return tmean, fmean, tstddev, fstddev

print("Datos de rango MEAN, STDDEV\n", setup4_range_stddev(main_df))

Datos de rango MEAN, STDDEV
 (0.9171880819366856, 0.7135169491525419, 1.0842932683540891, 0.91411294272203)


In [130]:
main_df

,open,high,low,close,tick_volume,max_zone_high,max_zone_low,POC,min_zone_high,min_zone_low,MIN,SETUP_1,SETUP_2,SETUP_3,SETUP_4,SETUP_4_range
key_0,,,,,,,,,,,,,,,,
2021.05.25,64.76,65.05,64.16,64.70,20950,64.91,64.86,64.87,64.63,64.17,64.21,True,True,True,True,0.11
2021.05.26,62.25,65.11,62.25,64.93,19551,64.81,64.75,64.91,64.58,62.25,62.25,True,False,False,True,2.66
2021.05.27,64.77,65.59,64.20,65.54,16575,64.51,64.48,65.28,65.27,64.79,64.20,False,False,False,True,0.51
2021.05.28,65.38,66.11,64.70,65.29,20234,65.65,65.61,65.63,65.56,64.70,64.70,True,True,True,True,0.25
2021.05.31,64.50,66.06,64.50,65.25,11688,65.75,65.73,65.74,65.58,65.41,64.50,True,True,True,True,1.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024.07.26,77.14,77.40,75.07,75.30,24615,77.07,77.04,75.51,77.03,76.30,75.77,False,False,False,True,1.63
2024.07.29,76.19,76.52,74.34,74.86,28905,74.81,74.75,76.04,75.91,74.86,76.51,True,True,True,False,0.15
2024.07.30,74.85,74.92,73.74,74.34,27580,74.00,73.95,73.96,74.90,74.40,74.84,False,False,False,False,0.89


In [131]:
def low_prob_MIN(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        poc = df.iloc[i]['MIN']

        if high >= poc >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YL(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yl = df.iloc[i]['low']

        if high >= yl >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YH(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yh = df.iloc[i]['high']

        if high >= yh >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100

def low_prob_YO(df):
    counter = 0

    for i in range(len(df.index) - 2):
        apertura = df.iloc[i+1]['open']
        high = df.iloc[i+1]['high']
        low = df.iloc[i+1]['low']
        yo = df.iloc[i]['open']

        if high >= yo >= low:
            counter = counter + 1

    return (counter / (len(df.index) - 1)) * 100


print("Porcentaje de test MIN Volume: ", low_prob_MIN(main_df))
print("Porcentaje de test YL: ", low_prob_YL(main_df))
print("Porcentaje de test YH: ", low_prob_YH(main_df))
print("Porcentaje de test YO: ", low_prob_YO(main_df))

Porcentaje de test MIN Volume:  59.84555984555985
Porcentaje de test YL:  45.559845559845556
Porcentaje de test YH:  56.62805662805663
Porcentaje de test YO:  48.51994851994852


In [132]:
main_df.to_csv("OUTPUT.csv")